In [2]:
import sys
import os
import os.path as osp
import argparse
import numpy as np
import cv2
import json
import torch
from glob import glob
from tqdm import tqdm
import torchvision.transforms as transforms
from torch.nn.parallel.data_parallel import DataParallel
import torch.backends.cudnn as cudnn

sys.path.insert(0, osp.join('..', 'main'))
sys.path.insert(0, osp.join('..', 'data'))
sys.path.insert(0, osp.join('..', 'common'))
from config import cfg
from model_new import get_model
from utils.preprocessing import load_img, process_bbox, generate_patch_image, get_iou
from utils.vis import vis_keypoints_with_skeleton, save_obj, render_mesh_orthogonal
from utils.mano import mano

In [3]:
cfg.set_args("0")
cudnn.benchmark = True

model_path = './snapshot_6.pth'
assert osp.exists(model_path), 'Cannot find model at ' + model_path
print('Load checkpoint from {}'.format(model_path))
model = get_model('test')
model = DataParallel(model).cuda()
ckpt = torch.load(model_path)
model.load_state_dict(ckpt['network'], strict=False)
model.eval()


>>> Using GPU: 0
Load checkpoint from ./snapshot_6.pth


C:\Users\vladi\AppData\Local\Temp\ipykernel_23388\1564869225.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path)


DataParallel(
  (module): Model(
    (body_backbone): ResNetBackbone(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [5]:
ir_image_dir = r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\RGB"
ir_image_paths = glob(osp.join(ir_image_dir, "*.jpg"))  # Assuming images are in .jpg forma
# Create a custom dataset for your infrared images
# Create a custom dataset for your infrared images
class InfraredDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]

        # Load and preprocess the image using InterWild's functions
        original_img = load_img(img_path)
        img_height, img_width = original_img.shape[:2]
        bbox = [0, 0, img_width, img_height]  # Use the entire image as the initial bbox
        bbox = process_bbox(bbox, img_width, img_height)
        img, img2bb_trans, bb2img_trans = generate_patch_image(original_img, bbox, 1.0, 0.0, False, cfg.input_img_shape)

        if self.transform:
            img = self.transform(img.astype(np.float32)) / 255
        
        return img

# Define transformations (match demo.py preprocessing)
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create dataset and dataloader
ir_dataset = InfraredDataset(ir_image_paths, transform=transform)
ir_dataloader = torch.utils.data.DataLoader(ir_dataset, batch_size=1, shuffle=True, num_workers=0)


In [7]:
# Calculate total size of model parameters in MB
total_params = sum(p.numel() * p.element_size() for p in model.parameters())
model_size_mb = total_params / (1024 * 1024)  # Convert bytes to MB
print(f"Model size: {model_size_mb:.2f} MB")


Model size: 221.18 MB


In [6]:
model.eval()
i = 0
for img_path in tqdm(ir_image_paths):
    i += 1
    file_name = img_path.split('/')[-1][:-4]
    
    # load image and make its aspect ratio follow cfg.input_img_shape
    original_img = load_img(img_path) 
    img_height, img_width = original_img.shape[:2]
    bbox = [0, 0, img_width, img_height]
    bbox = process_bbox(bbox, img_width, img_height)
    img, img2bb_trans, bb2img_trans = generate_patch_image(original_img, bbox, 1.0, 0.0, False, cfg.input_img_shape)
    transform = transforms.ToTensor()
    img = transform(img.astype(np.float32))/255
    img = img.cuda()[None,:,:,:]

    # forward to InterWild
    inputs = {'img': img}
    targets = {}
    meta_info = {}
    with torch.no_grad():
        out = model(inputs, targets, meta_info, 'test')
    
    vis_skeleton = original_img.copy()[:,:,::-1]
    for h in ('right', 'left'):
        # get outputs
        joint_img = out[h[0] + 'joint_img'].cpu().numpy()[0] # 2.5D pose
            # 2D skeleton
        joint_img_xy1 = np.concatenate((joint_img[:,:2], np.ones_like(joint_img[:,:1])),1)
        joint_img = np.dot(bb2img_trans, joint_img_xy1.transpose(1,0)).transpose(1,0)
        if h == 'right':
            color = (255,0,255) # purple
        else:
            color = (102,255,102) # green
            
        vis_skeleton = vis_keypoints_with_skeleton(vis_skeleton, joint_img, mano.sh_skeleton, color)
        print("hello")
        # save mesh
        #save_obj(mesh, mano.face[h], osp.join(mesh_save_path, file_name + '_' + h + '.obj'))

        # save MANO parameters
      
    # save box
    #cv2.imwrite(osp.join(render_save_path, file_name + '_box.jpg'), vis_box)

    # save 2D skeleton
    cv2.imwrite("modelnewtest.jpg", vis_skeleton)
    break

  0%|          | 0/80 [00:00<?, ?it/s]

hello
hello


  0%|          | 0/80 [00:03<?, ?it/s]
